# P-TEAM for novel mutations
In this tutorial, we will demonstrate how to predict with P-TEAM using a model for an individual TCR for which the binding of some mutations are unknown.

In [1]:
import numpy as np
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor

In [2]:
import sys
sys.path.append('../activation-prediction/')
from preprocessing import (add_activation_thresholds, full_aa_features,
                           get_aa_features, get_complete_dataset)
sys.path.append('../activation-prediction/active_learning')
from utils_al import get_aa_blosum

## Load the data
How the data is loaded will highly depend on your input data format. Here, we use the preprocessing functions that we used on our data. However, we will show how to format a minimal input required for training and prediction.

In [3]:
epitope = 'SIINFEKL'
base_activation = 83.52
df_data = add_activation_thresholds(get_complete_dataset(epitope), epitope=epitope)

# Select data from only one TCR and one experimental setting
df_data = df_data[df_data['tcr']=='OTI']
df_data = df_data[df_data['normalization']=='AS']
df_data = df_data[df_data['threshold']=='46.9']
df_data = df_data.reset_index(drop=True)

# Note that the base epitope should not be contained in the dataframe
df_data = df_data[df_data['epitope']!=epitope]

df_data = df_data[['epitope',
                   'activation',
                   'is_activated',
                  ]]
df_data.head(5)

,epitope,activation,is_activated
0,AIINFEKL,15.425218,False
1,CIINFEKL,8.974325,False
2,DIINFEKL,48.749267,True
3,EIINFEKL,5.924880,False
4,FIINFEKL,6.893491,False


From the `epitope` and the base epitope, we can create the additional columns `mut_pos`, `mut_ami`, and `orig_ami`, that are required as features.

In [4]:
df_data['mut_ami'] = df_data['epitope'].apply(lambda x: [x[i] for i in range(len(epitope)) 
                                                        if x[i] != epitope[i]][0])
df_data['mut_pos'] = df_data['epitope'].apply(lambda x: [i for i in range(len(epitope)) 
                                                        if x[i] != epitope[i]][0])
df_data['orig_ami'] = df_data['epitope'].apply(lambda x: [epitope[i] for i in range(len(epitope)) 
                                                         if x[i] != epitope[i]][0])
df_data.head(5)

,epitope,activation,is_activated,mut_ami,mut_pos,orig_ami
0,AIINFEKL,15.425218,False,A,0,S
1,CIINFEKL,8.974325,False,C,0,S
2,DIINFEKL,48.749267,True,D,0,S
3,EIINFEKL,5.924880,False,E,0,S
4,FIINFEKL,6.893491,False,F,0,S


As label, we require whether a APL activated the TCR (`is_activated`) for classification or the activation score (`activation`) for regression. From that, we can calculate the change of activation (`residual`). Directly predicting the activation score, will yield similar results.

In [5]:
df_data['residual'] = df_data['activation'] - base_activation

Finally, we randomly select 50% of the data as training and the rest as prediction data (unknown mutations without label).

In [6]:
np.random.seed(42)
mask_train = np.random.choice([True, False], size=len(df_data))
df_data_train = df_data[mask_train].copy()
df_data_pred = df_data[~mask_train].copy()
print('Training data: ', len(df_data_train))
print('Prediction data: ', len(df_data_pred))

Training data:  70
Prediction data:  82


## Calculate the features
If you provide the `df_data_train` and `df_data_pred` as described above, you can proceed with the remaining tutorial without adjustments.


First, we will load the features representing each amino acid by 5 so-called Atchley factors. Following, we will create a featurize representation of our data.

In [7]:
aa_features = get_aa_features()[['factors']]
aa_features.head()

feature_category factors                                
feature          factor0 factor1 factor2 factor3 factor4
A                 -0.591  -1.302  -0.733   1.570  -0.146
R                  1.538  -0.055   1.502   0.440   2.897
N                  0.945   0.828   1.299  -0.169   0.933
D                  1.050   0.302  -3.656  -0.259  -3.242
C                 -1.343   0.465  -0.862  -1.020  -0.255

In [8]:
df_features_train = full_aa_features(df_data_train, aa_features, base_peptide=epitope)
df_features_pred = full_aa_features(df_data_pred, aa_features, base_peptide=epitope)
df_features_pred.head()

,mut_pos,mut_ami$factors$factor0,mut_ami$factors$factor1,mut_ami$factors$factor2,mut_ami$factors$factor3,mut_ami$factors$factor4,orig_ami$factors$factor0,orig_ami$factors$factor1,orig_ami$factors$factor2,orig_ami$factors$factor3,...,diff_6$factors$factor0,diff_6$factors$factor1,diff_6$factors$factor2,diff_6$factors$factor3,diff_6$factors$factor4,diff_7$factors$factor0,diff_7$factors$factor1,diff_7$factors$factor2,diff_7$factors$factor3,diff_7$factors$factor4
1,0.0,-1.343,0.465,-0.862,-1.020,-0.255,-0.228,1.399,-4.76,0.67,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,-0.384,1.652,1.330,1.045,2.064,-0.228,1.399,-4.76,0.67,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,0.0,-1.019,-0.987,-1.505,1.266,-0.912,-0.228,1.399,-4.76,0.67,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
14,0.0,1.538,-0.055,1.502,0.440,2.897,-0.228,1.399,-4.76,0.67,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
16,0.0,-1.337,-0.279,-0.544,1.242,-1.262,-0.228,1.399,-4.76,0.67,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Prediction
### Classification

In [9]:
def predict_classification_score(x_train, y_train, x_pred, y_pred):
    clf = RandomForestClassifier(n_estimators=250,
                                 max_features='sqrt',
                                 random_state=42)
    clf = clf.fit(x_train, y_train['is_activated'])

    preds = clf.predict_proba(x_pred)
    preds = preds[:, (1 if preds.shape[1] == 2 else 0)]
    y_pred['class_score'] = preds
    return y_pred

In [10]:
df_pred = predict_classification_score(df_features_train, df_data_train, df_features_pred, df_data_pred)
df_pred.head()

,epitope,activation,is_activated,mut_ami,mut_pos,orig_ami,residual,class_score
1,CIINFEKL,8.974325,False,C,0,S,-74.545675,0.068
5,GIINFEKL,25.336532,False,G,0,S,-58.183468,0.096
9,LIINFEKL,1.168162,False,L,0,S,-82.351838,0.068
14,RIINFEKL,31.226973,False,R,0,S,-52.293027,0.076
16,VIINFEKL,3.493837,False,V,0,S,-80.026163,0.084


### Regression

In [11]:
def predict_regression_score(x_train, y_train, x_pred, y_pred):
    rfreg = RandomForestRegressor(n_estimators=250,
                                  max_features='sqrt',
                                  criterion='mae')
    rfreg = rfreg.fit(x_train, y_train['residual'])
    preds = rfreg.predict(x_pred)
    y_pred['reg_score'] = preds
    return y_pred

In [12]:
df_pred = predict_regression_score(df_features_train, df_data_train, df_features_pred, df_data_pred)
df_pred.head()

,epitope,activation,is_activated,mut_ami,mut_pos,orig_ami,residual,class_score,reg_score
1,CIINFEKL,8.974325,False,C,0,S,-74.545675,0.068,-66.999084
5,GIINFEKL,25.336532,False,G,0,S,-58.183468,0.096,-57.335624
9,LIINFEKL,1.168162,False,L,0,S,-82.351838,0.068,-66.853741
14,RIINFEKL,31.226973,False,R,0,S,-52.293027,0.076,-64.400582
16,VIINFEKL,3.493837,False,V,0,S,-80.026163,0.084,-66.242023


## Active Learning
### Suggesting new epitopes
The `uncertainty` is estimated as the smallest difference to the average prediction. Following, we select the `N` epitopes with the highest uncertainty.

In [13]:
def calculate_uncertainty(y_pred):   
    threshold = np.mean(y_pred['class_score'])
    y_pred['uncertainty'] = 1 - np.abs(y_pred['class_score'].values-threshold)
    return y_pred

In [14]:
df_pred = calculate_uncertainty(df_pred)
df_pred.head()

,epitope,activation,is_activated,mut_ami,mut_pos,orig_ami,residual,class_score,reg_score,uncertainty
1,CIINFEKL,8.974325,False,C,0,S,-74.545675,0.068,-66.999084,0.475024
5,GIINFEKL,25.336532,False,G,0,S,-58.183468,0.096,-57.335624,0.503024
9,LIINFEKL,1.168162,False,L,0,S,-82.351838,0.068,-66.853741,0.475024
14,RIINFEKL,31.226973,False,R,0,S,-52.293027,0.076,-64.400582,0.483024
16,VIINFEKL,3.493837,False,V,0,S,-80.026163,0.084,-66.242023,0.491024


In [15]:
N = 8
df_pred.sort_values('uncertainty', ascending=False)[:N]

,epitope,activation,is_activated,mut_ami,mut_pos,orig_ami,residual,class_score,reg_score,uncertainty
75,SIIYFEKL,20.277670,False,Y,3,N,-63.242330,0.496,-47.559921,0.903024
115,SIINFECL,31.844693,False,C,6,K,-51.675307,0.412,-45.657878,0.819024
120,SIINFEHL,74.341811,True,H,6,K,-9.178189,0.376,-44.171788,0.783024
108,SIINFRKL,51.837206,True,R,5,E,-31.682794,0.332,-43.575847,0.739024
126,SIINFEQL,35.839161,False,Q,6,K,-47.680839,0.320,-48.521750,0.727024
48,SINNFEKL,86.746918,True,N,2,I,3.226918,0.868,-20.483733,0.724976
117,SIINFEEL,10.194754,False,E,6,K,-73.325246,0.316,-44.170656,0.723024
45,SIKNFEKL,58.874642,True,K,2,I,-24.645358,0.876,-21.585093,0.716976


### Initial set
The initial epitopes were choosen as the APLs with the largest BLOSUM distance to the base epitope at each position and can be calculated as follows.

In [16]:
def blosum_distance(epitope_1, epitope_2):
    distance = 0
    blosum_mat = get_aa_blosum()
    for idx, (aa_1, aa_2) in enumerate(zip(epitope_1, epitope_2)):
        aa_distance = blosum_mat.loc[aa_1][aa_2]
        distance += aa_distance
    return distance

In [17]:
def init_blosum(base_epitope, all_apls):
    list_apls = []
    blosum_mat = get_aa_blosum()
    for idx, aa_base in enumerate(base_epitope):
        possible_aas = blosum_mat.loc[aa_base]
        aas_present = [aa for aa in possible_aas.index if 
                       base_epitope[:idx] + aa + base_epitope[idx+1:] in all_apls
                       and aa != aa_base]
        possible_aas = possible_aas[aas_present]
        if len(possible_aas) == 0:
            continue
        #print(possible_aas)
        new_aa = possible_aas.idxmin()
        new_epitope = base_epitope[:idx] + new_aa + base_epitope[idx+1:]
        list_apls.append(new_epitope)
    return list_apls

In [18]:
init_blosum(epitope, df_data['epitope'].values)

['WIINFEKL',
 'SGINFEKL',
 'SIGNFEKL',
 'SIIWFEKL',
 'SIINPEKL',
 'SIINFCKL',
 'SIINFECL',
 'SIINFEKD']